In [ ]:
import pandas as pd
import numpy as np
import math
import math
from Util import *
from CLT_class import CLT
from MIXTURE_CLT import MIXTURE_CLT
from __future__ import print_function

In [ ]:
datasetList = ["accidents","baudio","bnetflix","jester","kdd","msnbc","nltcs","plants","pumsb_star","tretail"]

data = {}
for i in datasetList:
    data[i+".ts.data"] = Util.load_dataset("datasets\\dataset\\"+i+".ts.data")
    data[i+".test.data"] = Util.load_dataset("datasets\\dataset\\"+i+".test.data")
    data[i+".valid.data"] = Util.load_dataset("datasets\\dataset\\"+i+".valid.data")

    

## Tree Bayesian networks

In [ ]:
#CLT

clt=CLT()
i = 0
likelihood = {}
probabilities = {}
items = list(data.keys())
while i < 30:
    clt.learn(data[items[i]])
    i += 1
    likelihood[items[i]] = clt.computeLL(data[items[i]])/data[items[i]].shape[0]
    i += 2


In [ ]:
print("Likelihood Values - ",likelihood)

## Mixtures of Tree Bayesian networks using EM:

#### Learning on Train data to find the best values of K to be used on test data

In [ ]:
kval = {}
items = list(data.keys())
ncomponents=10 #number of components
max_iter=50 #max number of iterations for EM
epsilon=1e-2 #converge if the difference in the log-likelihods between two iterations is smaller 1e-1

i = 0
kvals = [2,5,10,20]

while i < 30:
    kMixCLT = [MIXTURE_CLT() for p in kvals]
    p = 0
    temp = []
    print("Running for dataset - ", items[i])
    for k in kvals:
        t = i 
        # print([items[t]])
        kMixCLT[p].learn(data[items[t]],k,max_iter,epsilon)
        t += 2
        x = kMixCLT[p].computeLL(data[items[t]] , k)/data[items[t]].shape[0]
        print("LL for ", k , " - ", x  )
        temp.append(x)
        p += 1
        
    kval[items[i]] = [ kvals[temp.index(max(temp))] , max(temp)] 

    print(temp)
    print(kval)
    i += 3

#### Running on Test data

In [ ]:
items = list(data.keys())
ncomponents=10 #number of components
max_iter=50 #max number of iterations for EM
epsilon=1e-2 #converge if the difference in the log-likelihods between two iterations is smaller 1e-1


i = 0

while i < 30:
    kMixCLT = [MIXTURE_CLT() for p in kvals]
    p = 0
    temp = []
    print("Running for dataset - ", items[i])
    for l, k in kval[data[items[i]]]:
        t = i 
        # print([items[t]])
        kMixCLT[p].learn(data[items[t]],k,max_iter,epsilon)
        t += 1
        x = kMixCLT[p].computeLL(data[items[t]] , k)/data[items[t]].shape[0]
        print("LL for ", k , " - ", x  )
        temp.append(x)
        p += 1
        
    kval[items[i]] = [ kvals[temp.index(max(temp))] , max(temp)] 

    print(temp)
    print(kval)
    i += 3

## Mixtures of Tree Bayesian networks using Random Forests

In [ ]:
#Random Forest
kval = {}
items = list(data.keys())
# mix_clt=MIXTURE_CLT.MIXTURE_CLT()
ncomponents=10 #number of components
max_iter=50 #max number of iterations for EM
epsilon=1e-2 #converge if the difference in the log-likelihods between two iterations is smaller 1e-1

i = 0

kvals = [2,5,10,20]
rVals = [10, 20, 50, 100]
while i < 30:
    temp = []
    print("Running for dataset - ", items[i])
    for k in kvals:
        for r in rVals:
            print("K = ", str(k), " r = ", r)
            rf = MIXTURE_CLT()
            t = i 
            # print([items[t]])
            rf.learn(data[items[t]],k,max_iter,epsilon,r)
            t += 2
            x = rf.computeLL(data[items[t]] , k)
            temp.append([k,r,x])
            p += 1
        kval[items[i]+" "+str(k)] = temp 
    i += 3

In [ ]:
items = list(data.keys())
max_iter=50 #max number of iterations for EM
epsilon=1e-2 #converge if the difference in the log-likelihods between two iterations is smaller 1e-1

i = 0

kvals = [2]  ## Manually setting the best k value from above execution
rVals = [10] ## Manually setting the best r value from above execution

print("K = ", str(kvals[0]), " r = ", rVals[0])
rf = MIXTURE_CLT()
# print([items[t]])
rf.learn(data["accidents.ts.data"],kvals[0],max_iter,epsilon,rVals[0])
